In [ ]:
METADATA = {
    'title': 'ACCESS Resource Provider Report',
    'version': 1,
    'description': 'Summary of resource data for ACCESS Resource Providers.',
    'history': [
        [ '1', '2025-07-17', 'Initial Version.']
    ]
}

In [ ]:
from datetime import datetime, timedelta
from IPython.display import display, Markdown, HTML
import inspect
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio
import sys
import xdmod_data.themes
from xdmod_data.warehouse import DataWarehouse
from xdmod_data.report import header, footer, set_styles
import warnings
pd.set_option('future.no_silent_downcasting', True)
pio.renderers.default = 'plotly_mimetype+notebook'
pio.templates.default = 'timeseries'
def custom_warning_handler(message, category, filename, lineno, file=None, line=None):
    if file is None:
        file = sys.stderr
    print(f'{message}', file=file)
warnings.showwarning = custom_warning_handler

In [ ]:
# This cell contains function definitions for the various helper
# functions used by the notebook
# The functions assume that the following global variables are 
# defined:
#
# dw - DataWarehouse instance
# PROVIDER - str
# RESOURCES - dict containing the resource information
# TWO_YEARS_AGO_QUARTER_START_DATE - str start date for historical plots
# CURRENT_QUARTER_END_DATE - str end of the report period
# CURRENT_QUARTER_START_DATE - str start of the report period


def custom_warning_handler(message, category, filename, lineno, file=None, line=None):
    if file is None:
        file = sys.stderr
    print(f'{message}', file=file)


warnings.showwarning = custom_warning_handler


def get_quarter_dates(current_year, current_quarter):
    quarter_dates = []
    start_month = (current_quarter - 1) * 3 + 1
    y = current_year - 2
    m = start_month
    d = datetime(y, m, 1)
    still_going = True
    while still_going:
        if m == start_month and y == current_year:
            still_going = False
        if m == 10:
            y += 1
            m = 1
        else:
            m += 3
        start_date = d.strftime('%Y-%m-%d')
        d = datetime(y, m, 1)
        end_date = (d - timedelta(days=1)).strftime('%Y-%m-%d')
        quarter_dates.append([start_date, end_date])
    return quarter_dates


def no_performance_data_warning(resource):
    warnings.warn(f'The plot for {resource} cannot be generated; it requires performance data to be sent to the ACCESS Metrics team.')


def get_resources():
    with dw:
        resources = dw.get_resources(PROVIDER)
    return resources


def two_year_line_plot_melt_and_plot(
    df,
    y,
):
    df = pd.melt(
        df,
        id_vars=['Date'],
        var_name='Resource',
        value_name=y,
    )
    plot = px.line(
        df,
        x='Date',
        y=y,
        title=f'{y} by Resource by Quarter, Last Two Years',
        color='Resource',
        markers=True,
    )
    plot.update_xaxes(
        dtick='M3',
    )
    plot.update_traces(
        hovertemplate='%{y:,.0f}',
    )
    plot.update_layout(
        xaxis_tickformat='Q%q %Y',
        yaxis_tickformat=',',
        hovermode='x unified',
        hoverlabel_namelength=-1,
    )
    plot.show()
    display(figure_caption(f'{y} by Resource by Quarter, Last Two Years'))


def two_year_line_plot_by_resource(
    jobs_metric,
    cloud_dimension,
    y,
):
    df = None
    with dw:
        if len(RESOURCES['Jobs']) > 0:
            tmp_df = dw.get_data(
                duration=(TWO_YEARS_AGO_QUARTER_START_DATE, CURRENT_QUARTER_END_DATE),
                realm='Jobs',
                metric=jobs_metric,
                dimension='Resource',
                dataset_type='timeseries',
                aggregation_unit='Quarter',
                filters={
                    'Service Provider': PROVIDER,
                },
            )
            tmp_df = tmp_df.reset_index(names='Date')
            if df is None:
                df = tmp_df
        if len(RESOURCES['Cloud']) > 0:
            tmp_df = pd.DataFrame()
            for resource in RESOURCES['Cloud']:
                project_counts = []
                tmp_df_inner = dw.get_data(
                    duration=(TWO_YEARS_AGO_QUARTER_START_DATE, CURRENT_QUARTER_END_DATE),
                    realm='Cloud',
                    metric='CPU Hours: Total',
                    dimension=cloud_dimension,
                    dataset_type='timeseries',
                    aggregation_unit='Quarter',
                    filters={
                        'Resource': resource,
                    },
                )
                tmp_df_inner_transposed = tmp_df_inner.transpose()
                for quarter in tmp_df_inner_transposed.columns:
                    project_counts.append(
                        len(tmp_df_inner_transposed[tmp_df_inner_transposed[quarter] > 0]),
                    )
                tmp_df[resource] = project_counts
            tmp_df.index = tmp_df_inner.index
            tmp_df = tmp_df.reset_index(names='Date')
            if df is None:
                df = tmp_df
    two_year_line_plot_melt_and_plot(df, y)


def two_year_line_plot_by_quarter(
    y=None,
    resource=None,
    dimension=None,
    nlargest=0,
    vertical_legend=False,
):
    if y == 'projects':
        metric = 'Number of Allocations: Active'
        metric_label = 'Number of Active Projects'
    elif y == 'users':
        metric = 'Number of Users: Active'
        metric_label = 'Number of Active Users'
    elif y == 'ace':
        metric = 'ACCESS Credit Equivalents Charged: Total (SU)'
        metric_label = 'ACCESS Credit Equivalents Charged'
    if resource == 'all':
        dimension = dimension_label = 'Resource'
        filters = {
            'Service Provider': PROVIDER,
        }
    else:
        filters = {
            'Resource': resource,
        }
    if dimension == 'pfos':
        dimension = 'Parent Science'
        dimension_label = 'Parent Field of Science'
    elif dimension == 'academic status':
        dimension = 'User NSF Status'
        dimension_label = 'User Academic Status'
    elif dimension == 'project':
        dimension = 'Allocation'
        dimension_label = 'Project'
    with dw:
        df = dw.get_data(
            duration=(TWO_YEARS_AGO_QUARTER_START_DATE, CURRENT_QUARTER_END_DATE),
            realm=('Cloud' if len(RESOURCES['Cloud']) > 0 else 'Jobs'),
            metric=metric,
            dimension=dimension,
            dataset_type='timeseries',
            aggregation_unit='Quarter',
            filters=filters,
        )
    df = df.rename(
        columns={
            dimension: dimension_label,
        },
    )
    df = df.reset_index(names='Date')
    df = pd.melt(
        df,
        id_vars=['Date'],
        var_name=dimension_label,
        value_name=metric_label,
    )
    top_dimension_labels = None
    category_orders = None
    title = metric_label + (
        (f' on {resource}')
        if resource != 'all'
        else ''
    ) + f' by {dimension_label} by Quarter, Last Two Years'
    if nlargest > 0:
        top_dimension_labels = df[
            df['Date'] == CURRENT_QUARTER_START_DATE
        ].nlargest(
            nlargest,
            metric_label,
        )[dimension_label].tolist()
        df = df[df[dimension_label].isin(top_dimension_labels)]
        category_orders = {
            dimension_label: top_dimension_labels,
        }
        title += f', Top {nlargest}'
    plot = px.line(
        df,
        x='Date',
        y=metric_label,
        title=title,
        color=dimension_label,
        markers=True,
        category_orders=category_orders,
    )
    plot.update_traces(
        hovertemplate='%{y:,.0f}',
    )
    plot.update_layout(
        xaxis_tickformat='Q%q %Y',
        yaxis_tickformat=',',
        hovermode='x unified',
        hoverlabel_namelength=-1,
    )
    if vertical_legend:
        plot.update_layout(
            legend_orientation='v',
            legend_xanchor='left',
            legend_x=0,
            legend_yanchor='bottom',
            legend_y=-1.3,
        )
    plot.show()
    display(figure_caption(title))
    return top_dimension_labels


def make_cpu_user_histogram(resource):
    no_data = False
    try:
        with dw:
            cpu_user_histogram_df = dw.get_data(
                duration=(CURRENT_QUARTER_START_DATE, CURRENT_QUARTER_END_DATE),
                realm='SUPREMM',
                metric='CPU Hours: Total',
                dimension='CPU User Value',
                dataset_type='aggregate',
                filters={
                    'Resource': resource,
                },
            )
    except KeyError:
        no_data = True
    if no_data:
        no_performance_data_warning(resource)
        return
    plot = px.bar(
        cpu_user_histogram_df,
        y='CPU Hours: Total',
        title=f'CPU Usage by CPU User Value on {resource}',
    )
    plot.update_traces(
        hovertemplate='%{x}<br>CPU Hours: Total: <b>%{y:,.0f}</b>',
    )
    plot.show()
    display(figure_caption(f'CPU Usage by CPU User Value on {resource}'))


def make_cpu_efficiency_plot(resource):
    metrics = ['avg_percent_cpu_idle', 'wall_time']
    efficiency_series = {}
    no_data = False
    try:
        with dw:
            for metric in metrics:
                efficiency_series[metric] = dw.get_data(
                    duration=(CURRENT_QUARTER_START_DATE, CURRENT_QUARTER_END_DATE),
                    realm='SUPREMM',
                    metric=metric,
                    dimension='User',
                    dataset_type='aggregate',
                    filters={
                        'Resource': resource,
                    },
                ).reset_index()
                if len(efficiency_series[metric]) == 0:
                    no_data = True
    except KeyError:
        no_data = True
    if no_data:
        no_performance_data_warning(resource)
        return
    cpu_usage_df = pd.merge(
        efficiency_series['avg_percent_cpu_idle'],
        efficiency_series['wall_time'],
        on='User',
        how='outer',
    ).dropna(how='any')
    avg_cpu_str = 'Avg CPU %: Idle: weighted by core-hour'
    cpu_hours_str = 'CPU Hours: Total'
    top_cpu_hours_total = cpu_usage_df['CPU Hours: Total'].max()
    half_top_cpu_hours_total = 0.5 * top_cpu_hours_total
    cpu_usage_df['In First Quadrant'] = cpu_usage_df.apply(
        lambda row: row[avg_cpu_str] > 50 and row[cpu_hours_str] > half_top_cpu_hours_total,
        axis=1,
    )
    plot = px.scatter(
        cpu_usage_df,
        title=f'CPU Usage on {resource} by User',
        x=avg_cpu_str,
        y=cpu_hours_str,
        color='In First Quadrant',
        hover_data=['User'],
    )
    plot.add_vline(x=50, line_width=1)
    plot.add_hline(y=int(0.5 * top_cpu_hours_total), line_width=1)
    plot.update_layout(showlegend=False)
    hovertemplate = inspect.cleandoc(
        f"""
        <b>%{{customdata[0]}}</b><br>
        {avg_cpu_str}: <b>%{{x:.0f}} %</b><br>
        {cpu_hours_str}: <b>%{{y:,.0f}} core hours</b>
        <extra></extra>
        """,
    )
    plot.update_traces(hovertemplate=hovertemplate)
    plot.show()
    display(figure_caption(f'CPU Usage on {resource} by User'))


def get_science_gateway_usage():
    try:
        with dw:
            df = dw.get_data(
                duration=(TWO_YEARS_AGO_QUARTER_START_DATE, CURRENT_QUARTER_END_DATE),
                realm='Gateways',
                metric='Number of Gateway Users: Active (Number of Users)',
                dimension='Resource',
                dataset_type='timeseries',
                aggregation_unit='Quarter',
                filters={
                    'Service Provider': PROVIDER,
                },
            )
            df = df.reset_index(names='Date')
        return df
    except KeyError:
        return None


def construct_cloud_df(dimension, reslist):
    df = pd.DataFrame()
    for resource in reslist:
        project_counts = []
        tmp_df = dw.get_data(
            duration=(TWO_YEARS_AGO_QUARTER_START_DATE, CURRENT_QUARTER_END_DATE),
            realm='Cloud',
            metric='CPU Hours: Total',
            dimension=dimension,
            dataset_type='timeseries',
            aggregation_unit='Quarter',
            filters={
                'Resource': resource,
            },
        )
        tmp_df_transposed = tmp_df.transpose()
        for quarter in tmp_df_transposed.columns:
            project_counts.append(
                len(tmp_df_transposed[tmp_df_transposed[quarter] > 0]),
            )
        df[resource] = project_counts
    df.index = tmp_df.index
    return df


def get_report_data():
    REALMS = ['Jobs', 'Cloud']
    STATISTICS = ['projects', 'users']
    GROUP_BYS = ['all', 'pfos']
    dfs = {}
    resources = {}
    for realm in REALMS:
        dfs[realm] = {}
        resources[realm] = []
        for statistic in STATISTICS:
            dfs[realm][statistic] = {
                'all': {},
            }
    with dw:
        has_active_jobs = True
        try:
            jobs_resources_df = dw.get_data(
                duration=(CURRENT_QUARTER_START_DATE, CURRENT_QUARTER_END_DATE),
                realm='Jobs',
                metric='ACCESS Credit Equivalents Charged: Total (SU)',
                dimension='Resource',
                dataset_type='timeseries',
                aggregation_unit='Quarter',
                filters={
                    'Service Provider': PROVIDER,
                },
            )
        except KeyError:
            has_active_jobs = False
        if has_active_jobs:
            resources['Jobs'] = jobs_resources_df.columns.tolist()
            dfs['Jobs']['projects']['all']['all'] = dw.get_data(
                duration=(TWO_YEARS_AGO_QUARTER_START_DATE, CURRENT_QUARTER_END_DATE),
                realm='Jobs',
                metric='Number of Allocations: Active',
                dimension='Resource',
                dataset_type='timeseries',
                aggregation_unit='Quarter',
                filters={
                    'Service Provider': PROVIDER,
                },
            )
            for statistic in STATISTICS:
                for resource in ['all'] + resources['Jobs']:
                    dfs['Jobs'][statistic][resource] = {}
                    for group_by in GROUP_BYS:
                        if (
                            group_by == 'all' and resource != 'all'
                            or group_by != 'all' and resource == 'all'
                        ):
                            continue
                        if statistic == 'projects':
                            metric = 'Number of Allocations: Active'
                        elif statistic == 'users':
                            metric = 'Number of Users: Active'
                        elif statistic == 'ace':
                            metric = 'ACCESS Credit Equivalents Charged: Total (SU)'
                        if resource == 'all':
                            dimension = 'Resource'
                            filters = {
                                'Service Provider': PROVIDER,
                            }
                        else:
                            filters = {
                                'Resource': resource,
                            }
                        if group_by == 'pfos':
                            dimension = 'Parent Science'
                        elif group_by == 'academic status':
                            dimension = 'User NSF Status'
                        elif group_by == 'project':
                            dimension = 'Allocation'
                        dfs['Jobs'][statistic][resource][group_by] = dw.get_data(
                            duration=(TWO_YEARS_AGO_QUARTER_START_DATE, CURRENT_QUARTER_END_DATE),
                            realm='Jobs',
                            metric=metric,
                            dimension=dimension,
                            dataset_type='timeseries',
                            aggregation_unit='Quarter',
                            filters=filters,
                        )

    dfs['Cloud']['cpu hours'] = {
        'all': {},
    }
    with dw:
        has_active_cloud_vms = True
        try:
            jobs_resources_df = dw.get_data(
                duration=(CURRENT_QUARTER_START_DATE, CURRENT_QUARTER_END_DATE),
                realm='Cloud',
                metric='CPU Hours: Total',
                dimension='Resource',
                dataset_type='timeseries',
                aggregation_unit='Quarter',
                filters={
                    'Provider': PROVIDER,
                },
            )
        except KeyError:
            has_active_cloud_vms = False
        if has_active_cloud_vms:
            resources['Cloud'] = jobs_resources_df.columns.tolist()
            dfs['Cloud']['projects']['all']['all'] = construct_cloud_df('Allocation', resources['Cloud'])
            dfs['Cloud']['users']['all']['all'] = construct_cloud_df('User', resources['Cloud'])

    return dfs, resources


def summarize_usage(report_data, resources):
    top_5 = {'projects': {}}
    top_5_strings = {}
    for resource in resources['Jobs']:
        top_5['projects'][resource] = {
            'pfos': {},
        }
        top_5['projects'][resource]['pfos'] = report_data['Jobs']['projects'][resource]['pfos'].loc[CURRENT_QUARTER_START_DATE].nlargest(5)
        top_5_strings[resource] = []
        for pfos, count in top_5['projects'][resource]['pfos'].items():
            top_5_strings[resource].append(f'{pfos} ({int(count)} projects)')
    return top_5_strings


def get_ondemand_usage():
    try:
        with dw:
            df = dw.get_data(
                duration=(TWO_YEARS_AGO_QUARTER_START_DATE, CURRENT_QUARTER_END_DATE),
                realm='OnDemand',
                metric=('Number of Users: Active' if PROVIDER == 'SDSC' else 'Number of Known ACCESS Users: Active (Number of Users)'),
                dimension='Resource',
                dataset_type='timeseries',
                aggregation_unit='Quarter',
                filters={
                    'Service Provider': PROVIDER,
                },
            )
            df = df.reset_index(names='Date')
        return df

    except KeyError:
        return None

def get_jobs_utilization(resource):
    metrics = [
        ('Jobs', 'ACCESS Credit Equivalents Charged: Total (SU)'),
        ('Resource Specifications', 'ACCESS Credit Equivalents Available: Total (SU)'),
        ('Resource Specifications', 'ACCESS Credit Equivalents Available: Allocated (SU)')

    ]

    data = []
    with dw:
        for (realm, metric) in metrics:
            data.append(dw.get_data(
                duration=(TWO_YEARS_AGO_QUARTER_START_DATE, CURRENT_QUARTER_END_DATE),
                realm=realm,
                metric=metric,
                dataset_type='timeseries',
                aggregation_unit='Quarter',
                filters={'Resource': [resource]},
            ))
            
    return pd.concat(data, axis=1).rename(columns = {'ACCESS Credit Equivalents Charged: Total (SU)': 'Used',
                                                     'ACCESS Credit Equivalents Available: Allocated (SU)': 'Available for ACCESS',
                                                     'ACCESS Credit Equivalents Available: Total (SU)': 'Resource Limit'})


def plot_jobs_utilization(resource):
    
    data = get_jobs_utilization(resource)
    fig = px.line(data,
                  markers=True,
                  labels = {'value': 'ACCESS Credit Equivalents', 'Metric': ''})
    fig.update_xaxes(
        dtick='M3',
    )
    fig.update_traces(
        hovertemplate='%{y:,.0f}',
    )
    fig.update_layout(
        xaxis_tickformat='Q%q %Y',
        hovermode='x unified',
        hoverlabel_namelength=-1,
    )
    fig['data'][2]['line']['dash'] = 'dot'
    fig.show()
    display(figure_caption(f'''Usage of {resource} from {TWO_YEARS_AGO_QUARTER_START_DATE}
    to {CURRENT_QUARTER_END_DATE}. Usage is shown converted to ACCESS Credit Equivalents. 
    The red solid line shows the hypothetical upper bound for credit usage based on the 
    number of compute elements on the resource. The grey dotted line shows the mean value 
    of the ACCESS Credits that the resource is obliged to make available for use by ACCESS
    allocated jobs.'''))


SECTION_NUMBER = 0
FIGURE_NUMBER = 0


def figure_caption(caption):
    global FIGURE_NUMBER
    FIGURE_NUMBER += 1
    return Markdown(f'_Figure {SECTION_NUMBER}.{FIGURE_NUMBER} {caption}._')


def section_title(description):
    global SECTION_NUMBER
    SECTION_NUMBER += 1
    global FIGURE_NUMBER
    FIGURE_NUMBER = 0
    return Markdown(f'# {SECTION_NUMBER}. {description}')


In [ ]:
# This cell is for debugging
if 'RP' not in os.environ:
    os.environ['RP'] = 'IU - Indiana University'
    #os.environ['RP'] = 'Purdue'
    #os.environ['RP'] = 'NCSA'
if 'YEAR' not in os.environ:
    os.environ['YEAR'] = '2025'
if 'QUARTER' not in os.environ:
    os.environ['QUARTER'] = '2'

In [ ]:
PROVIDER = os.environ['RP']
YEAR = int(os.environ['YEAR'])
QUARTER = int(os.environ['QUARTER'])

In [ ]:
QUARTER_DATES = get_quarter_dates(YEAR, QUARTER)
TWO_YEARS_AGO_QUARTER_START_DATE = QUARTER_DATES[0][0]
CURRENT_QUARTER_START_DATE, CURRENT_QUARTER_END_DATE = QUARTER_DATES[-1]

In [ ]:
dw = DataWarehouse()
dfs, RESOURCES = get_report_data()
top_5_strings = summarize_usage(dfs, RESOURCES)

In [ ]:
# Introduction and data summary

display(set_styles())
display(header())

toc =  f'''# ACCESS Metrics Report for {PROVIDER}
{CURRENT_QUARTER_START_DATE} through {CURRENT_QUARTER_END_DATE}

## Table of contents

- [1. Introduction](#1.-Introduction)
- [2. Resource Utilization](#2.-Resource-Utilization)
- [3. Active projects](#3.-Active-projects)
- [4. Active users](#4.-Active-users)
- [5. Open OnDemand Usage](#5.-Open-OnDemand-Usage)
- [6. Science Gateway Usage](#6.-Science-Gateway-Usage)
'''

if len(RESOURCES['Jobs']) > 0:
    toc += '- [7. Job Performance](#7.-Job-Performance)'

display(Markdown(toc))
display(section_title('Introduction'))
display(Markdown(
    f'''This report from the ACCESS Metrics team contains information about the utilization and performance of the resources
    from the Resource Provider **{PROVIDER}**
    for the dates {CURRENT_QUARTER_START_DATE} through {CURRENT_QUARTER_END_DATE}.
    The data in this report come from ACCESS XDMoD via the ACCESS Central Database as well as data provided directly to the
    ACCESS Metrics team by the Resource Provider.
    You can contact the ACCESS Metrics team regarding the data in this report by [submitting a ticket](https://support.access-ci.org/help-ticket).
    Below is a summary of the data for each active resource during this time period.
    '''
))
if len(RESOURCES['Jobs']) > 0:
    display(Markdown('### Compute Resources'))
    for resource in RESOURCES['Jobs']:
        display(Markdown(f'#### {resource}'))
        display(Markdown(
            f'''- There were **{int(dfs['Jobs']['projects']['all']['all'].at[CURRENT_QUARTER_START_DATE, resource]):,} active projects**
            (projects that ran at least one job).
            \n- There were **{int(dfs['Jobs']['users']['all']['all'].at[CURRENT_QUARTER_START_DATE, resource]):,} active users**
            (users who ran at least one job).
            \n- The top 5 parent fields of science in terms of number of active projects were:
            '''
            + ''.join([f'\n\n    - **{string}**' for string in top_5_strings[resource]])
        ))
if len(RESOURCES['Cloud']) > 0:
    display(Markdown('### Cloud Resources'))
    for resource in RESOURCES['Cloud']:
        display(Markdown(f'#### {resource}'))
        display(Markdown(
            f'''- There were **{int(dfs['Cloud']['projects']['all']['all'].at[CURRENT_QUARTER_START_DATE, resource]):,} active projects**
            (projects that ran at least one cloud virtual machine).
            \n- There were **{int(dfs['Cloud']['users']['all']['all'].at[CURRENT_QUARTER_START_DATE, resource]):,} active users**
            (users that ran at least one cloud virtual machine).
            '''
        ))

In [ ]:
# Information about Resource Utilization

display(section_title('Resource Utilization'))

if len(RESOURCES['Jobs']):
    display(Markdown("""
This section shows utilization information for ACCESS-allocated compute resources. 
To faciliate comparison across different types of compute resources (such as CPU or GPU),
usage is presented in ACCESS Credit Equivalents. Each compute resource has a conversion
factor between a resource service unit and an ACCESS Credit. For example, one ACCESS equivalent
credit is equal to 1 CPU core hour on SDSC Expanse (which uses AMD EPYC 7742 processors).
    
The data here are based on information reported to ACCESS via the Usage Reporting API.
    """))

    for resource in RESOURCES['Jobs']:
        plot_jobs_utilization(resource)

In [ ]:
# Information about active projects

display(section_title('Active projects'))

two_year_line_plot_by_resource(
    jobs_metric='Number of Allocations: Active',
    cloud_dimension='Allocation',
    y='Number of Active Projects',
)

if len(RESOURCES['Jobs']) > 0:
    display(Markdown('### By Parent Field of Science'))

for resource in RESOURCES['Jobs']:
    two_year_line_plot_by_quarter(
        y='projects',
        resource=resource,
        dimension='pfos',
    )

In [ ]:
# Information about active users

display(section_title('Active users'))

two_year_line_plot_by_resource(
    jobs_metric='Number of Users: Active',
    cloud_dimension='User',
    y='Number of Active Users',
)

for resource in RESOURCES['Jobs']:
    two_year_line_plot_by_quarter(
        y='users',
        resource=resource,
        dimension='pfos',
    )

for resource in RESOURCES['Jobs']:
    two_year_line_plot_by_quarter(
        y='users',
        resource=resource,
        dimension='academic status',
    )

In [ ]:
# Information about Open OnDemand usage

display(section_title('Open OnDemand Usage'))

ondemand_usage_df = get_ondemand_usage()
if ondemand_usage_df is not None:
    two_year_line_plot_melt_and_plot(ondemand_usage_df, 'Number of Open OnDemand Users')
else:
    warnings.warn(
            'This plot cannot be generated; it requires Open OnDemand data to be sent to the ACCESS Metrics team.'
        )

In [ ]:
# Information about science gateways

display(section_title('Science Gateway Usage'))
science_gw_users_df = get_science_gateway_usage()
if science_gw_users_df is not None:
    two_year_line_plot_melt_and_plot(science_gw_users_df, 'Number of Science Gateway Users')
else:
    warnings.warn(f'Science Gateway usage not available for resources from {PROVIDER}')

In [ ]:
if len(RESOURCES['Jobs']) > 0:
    display(section_title('Job Performance'))
    
    display(Markdown('### CPU Usage by User'))
    for resource in RESOURCES['Jobs']:
        make_cpu_efficiency_plot(resource)
    display(Markdown(
"""
Click [here](https://xdmod.access-ci.org/#main_tab_panel:efficiency) to view more details in ACCESS XDMoD.
The link will take you to the Efficiency tab, which gives a program-wide view of all ACCESS-allocated compute resources.
There, you can recreate the plots from this report by choosing the "CPU Usage" chart and applying a resource filter.
"""
    ))

    display(Markdown('### CPU Usage by CPU User Value'))
    display(Markdown('CPU User Value is measured as the ratio of user CPU time to total CPU time for the cores that the job was assigned.'))
    for resource in RESOURCES['Jobs']:
        make_cpu_user_histogram(resource)

In [ ]:
footer(METADATA)